#**FID score**

In [ ]:
# Install required libraries
!pip install tensorflow==2.5.0
!pip install numpy
!pip install git+https://github.com/mseitzer/pytorch-fid.git


ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1)
ERROR: No matching distribution found for tensorflow==2.5.0
  Cloning https://github.com/mseitzer/pytorch-fid.git to /tmp/pip-req-build-8ddiak49
  Running command git clone --filter=blob:none --quiet https://github.com/mseitzer/pytorch-fid.git /tmp/pip-req-build-8ddiak49
  Resolved https://github.com/mseitzer/pytorch-fid.git to commit 0a754fb8e66021700478fd365b79c2eaa316e31b
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-fid: filename=pytorch_fid-0.3.0-py3-none-any.whl size=15484 sha25

In [ ]:
pip install tensorflow

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder, filename)
            img = image.load_img(img_path, target_size=(299, 299))
            img_array = image.img_to_array(img)
            images.append(img_array)
    return np.array(images)

def preprocess_images(images):
    processed_images = preprocess_input(images)
    return processed_images

def extract_features(model, images):
    features = model.predict(images)
    return features

def calculate_fid(real_features, generated_features):
    mu_real, sigma_real = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu_generated, sigma_generated = np.mean(generated_features, axis=0), np.cov(generated_features, rowvar=False)

    # Ensure positive definiteness for covariance matrices
    epsilon = 1e-6
    sigma_real += epsilon * np.eye(sigma_real.shape[0])
    sigma_generated += epsilon * np.eye(sigma_generated.shape[0])

    sigma_sqrt = sqrtm(sigma_real @ sigma_generated)
    fid = np.sum((mu_real - mu_generated)**2) + np.trace(sigma_real + sigma_generated - 2 * sigma_sqrt)
    return fid

# Set the paths to your real and generated image folders
real_images_path = "/content/dataset"
generated_images_path = "/content/generate"

# Load InceptionV3 model
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

# Load and preprocess images
real_images = load_images_from_folder(real_images_path)
generated_images = load_images_from_folder(generated_images_path)

real_images_processed = preprocess_images(real_images)
generated_images_processed = preprocess_images(generated_images)

# Extract features
real_features = extract_features(inception_model, real_images_processed)
generated_features = extract_features(inception_model, generated_images_processed)

# Calculate FID
fid_score = calculate_fid(real_features, generated_features)

print(f"FID Score: {fid_score}")


1/1 [==============================] - 3s 3s/step
FID Score: 384.1173296048412


# **IS score**

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

def compute_inception_score(folder_path, target_size=(299, 299), num_images=20):
    """ step 1 : Preprocessing images (resizing + normalizing) """
    # Get a list of all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    # Placeholder for preprocessed images
    preprocessed_images = []

    for i in range(min(num_images, len(image_files))):
        image_path = os.path.join(folder_path, image_files[i])

        # Read the image using OpenCV
        img = cv2.imread(image_path)

        # Resize the image
        img_resized = cv2.resize(img, target_size)

        # Normalize the pixel values to the range [0, 1]
        img_normalized = img_resized / 255.0

        preprocessed_images.append(img_normalized)

    # Convert the list to a NumPy array
    preprocessed_images = np.array(preprocessed_images)

    """ step 2 : Calculate IS Score """
    # Load InceptionV3 model
    inception_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')

    # Compute Inception Score
    synthetic_features = inception_model.predict(preprocess_input(np.array(preprocessed_images)))
    probabilities = np.exp(np.mean(np.log(synthetic_features + 1e-10), axis=1))
    is_score = np.mean(probabilities)

    return is_score


# Specify the path to the folder containing the images
folder_path = "/content/generated_images"

# Compute IS score
is_score = compute_inception_score(folder_path)

print(f"Inception Score (IS): {is_score}")